In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib as mpl

# imports custom RNCRN functions that are used throughout multiple  notebooks 
import os
import sys
from pathlib import Path

module_path = str(Path(os.path.abspath('')).parent)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.target_systems import *
from functions.RNCRN_tools import * 
from functions.RNCRN_phase_plane import *
from functions.create_attractors_tools import * 

In [2]:
# name of model to plot
filename = 'models/example_torus' 

# call a function to open the saved model and unpack the parameters to their named matrices 
params, number_of_exec_species, number_of_chemical_perceptrons = unpack_mat_model_rncrn(filename+'.mat')
alpha_mat, omega_mat, bias_vec, beta_vec, gamma_vec, tau_vec = params

# sets a time-scale (\mu parameter) for the chemical reaction network 
time_scale = 0.01

# sets all initial conditions to zero (overwritten later)
inits = np.zeros(number_of_exec_species+number_of_chemical_perceptrons)
crn_file = 'CRNs/torus_crn.txt'

# sets file location to store the compiled chemical reaction network
save_crn(crn_file, convert_RNCRN_params_to_CRN(params, time_scale, inits) )

# loads the chemical reaction network file from .txt and extracts the kinetic and stoichiometric data
species, reaction_rates, react_stoch, prod_stoch, stoch_mat, number_species, number_reactions, initial_concs = read_crn_txt(crn_file)

# creates a tuple of kinetic and stoichiometric data for the trajectory simulation 
args_tup_rncrn = (reaction_rates, react_stoch, stoch_mat,)

In [3]:
# sets the (x_1,x_2)-space boundaries (for visualisation  of the vector field)
lower_limit = 0.5
upper_limit = 3.5
step_size = 0.1

In [4]:
# Invokes the code for Algorithm 1 (to generate vector field)
# sets the parameter \delta
distance = 0.01

# sets the parameter \eta
magnitude = 1

# sets the parameter K (number of padding points)
number_of_padding_each_side = 5

# sets the interval of the variable that is used in the parametric equation to create a line of points 
low_parametric = 0
upper_parametric = 2*np.pi
step_parametric = 0.01

In [7]:
# numerical integration of the RNCRN 
# the time length of the trajectory
total_time = 205

# sets time steps
t_steps = float(5)
n_steps = int(total_time/t_steps)
min_step = 0.01
n_mini_steps = int(t_steps/min_step)
time_start = 0
time_finsh = t_steps

# handler function for setting new initial conditions in chemical reaction networks 
inits_rncrn, id_dict = new_initial_conditions(initial_concs, species, {'X_1':10.126, 'X_2': 4.257, 'X_3': 1.703})
sol_rncrn = solve_ivp(stoch_mat_to_mass_action, [time_start, time_finsh], inits_rncrn, args=args_tup_rncrn, rtol=10e-8, t_eval=np.arange(time_start, time_finsh,0.01))
res_x1 = sol_rncrn.y[id_dict['X_1'],:]
res_x2 = sol_rncrn.y[id_dict['X_2'],:]
res_x3 = sol_rncrn.y[id_dict['X_3'],:]

#sets a constant time interval for the numerical integration
t_evals=np.arange(time_start, total_time, min_step)

# simulates the time-trajectory of the chemical reaction netwrok 
for t_step in np.arange(1, n_steps, 1):
    t_eval = t_evals[n_mini_steps*t_step:n_mini_steps*(t_step+1)]
    time_start = t_eval[0]
    time_finsh = t_eval[-1]
    print('time = [', time_start, ',', time_finsh, ')')
    new_inits = sol_rncrn.y[:,-1]
    sol_rncrn = solve_ivp(stoch_mat_to_mass_action, [time_start,time_finsh], new_inits, args=args_tup_rncrn, rtol=10e-8, t_eval=t_eval)
    
    res_x1 = np.concatenate([res_x1, sol_rncrn.y[id_dict['X_1'],:]])
    res_x2 = np.concatenate([res_x2, sol_rncrn.y[id_dict['X_2'],:]])
    res_x3 = np.concatenate([res_x3, sol_rncrn.y[id_dict['X_3'],:]])

time = [ 5.0 , 9.99 )
time = [ 10.0 , 14.99 )
time = [ 15.0 , 19.990000000000002 )
time = [ 20.0 , 24.990000000000002 )
time = [ 25.0 , 29.990000000000002 )
time = [ 30.0 , 34.99 )
time = [ 35.0 , 39.99 )
time = [ 40.0 , 44.99 )
time = [ 45.0 , 49.99 )
time = [ 50.0 , 54.99 )
time = [ 55.0 , 59.99 )
time = [ 60.0 , 64.99 )
time = [ 65.0 , 69.99 )
time = [ 70.0 , 74.99 )
time = [ 75.0 , 79.99 )
time = [ 80.0 , 84.99 )
time = [ 85.0 , 89.99 )
time = [ 90.0 , 94.99 )
time = [ 95.0 , 99.99000000000001 )
time = [ 100.0 , 104.99000000000001 )
time = [ 105.0 , 109.99000000000001 )
time = [ 110.0 , 114.99000000000001 )
time = [ 115.0 , 119.99000000000001 )
time = [ 120.0 , 124.99000000000001 )
time = [ 125.0 , 129.99 )
time = [ 130.0 , 134.99 )
time = [ 135.0 , 139.99 )
time = [ 140.0 , 144.99 )
time = [ 145.0 , 149.99 )
time = [ 150.0 , 154.99 )
time = [ 155.0 , 159.99 )
time = [ 160.0 , 164.99 )
time = [ 165.0 , 169.99 )
time = [ 170.0 , 174.99 )
time = [ 175.0 , 179.99 )
time = [ 180.0 , 18

In [10]:
save = False
if save:
    np.savetxt("data/torus_traj_data.csv", np.asarray([ t_evals, res_x1, res_x2, res_x3]), delimiter=",")

In [5]:
alpha_mat, omega_mat, bias_vec, beta, gamma, tau = params
print('-print parameter values-')
print('original RNCRN')

print('omega_mat_0')
for val in omega_mat[:,0]:
    print(f'{val:.3f}', '\\\\')

print('omega_mat_1')
for val in omega_mat[:,1]:
    print(f'{val:.3f}', '\\\\')

print('alpha_mat_0')
for val in alpha_mat[0,:]:
    print(f'{val:.3f}', '\\\\')

print('alpha_mat_1')
for val in alpha_mat[1,:]:
    print(f'{val:.3f}', '\\\\')

print('bias_vec')
for val in bias_vec[:,0]:
    print(f'{val:.3f}', '\\\\')
    
print('beta')
for val in beta[:,0]:
    print(f'{val:.3f}', '\\\\')

print('gamma')
for val in gamma[:,0]:
    print(f'{val:.3f}', '\\\\')
    
print('tau')
for val in tau[:,0]:
    print(f'{val:.3f}', '\\\\')

-print parameter values-
original RNCRN
omega_mat_0
-39.116 \\
-17.292 \\
-3.625 \\
7.586 \\
-9.729 \\
207.473 \\
-109.965 \\
omega_mat_1
-2.949 \\
-1.592 \\
-36.064 \\
-105.612 \\
-9.325 \\
-327.333 \\
-28.268 \\
alpha_mat_0
-5.227 \\
-6.586 \\
0.332 \\
-1.297 \\
1.037 \\
-0.752 \\
7.607 \\
alpha_mat_1
-0.755 \\
-5.934 \\
-3.663 \\
14.209 \\
1.077 \\
1.567 \\
2.768 \\
bias_vec
292.047 \\
-280.508 \\
261.422 \\
-255.093 \\
262.072 \\
-198.768 \\
-139.727 \\
beta
1.570 \\
0.408 \\
6.885 \\
gamma
5.488 \\
7.535 \\
3.963 \\
9.258 \\
6.007 \\
0.004 \\
5.764 \\
tau
666.189 \\
812.839 \\
496.655 \\
912.664 \\
428.803 \\
713.388 \\
4637.415 \\
